<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/caption/ugen_image_captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/ugen-image-captioning-hf
%cd /content/ugen-image-captioning-hf
!pip install -q gradio uform

In [3]:
# cpu
%cd /content/ugen-image-captioning-hf
!python app.py

In [4]:
%%writefile /content/ugen-image-captioning-hf/app_cuda.py
import gradio as gr
from uform import gen_model
from PIL import Image
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model and processor
model = gen_model.VLMForCausalLM.from_pretrained("unum-cloud/uform-gen").to(device)
processor = gen_model.VLMProcessor.from_pretrained("unum-cloud/uform-gen")

def generate_caption(image, prompt):
    # Process the image and the prompt
    inputs = processor(texts=[prompt], images=[image], return_tensors="pt").to(device)

    # Generate the output
    with torch.inference_mode():
        output = model.generate(
            **inputs,
            do_sample=False,
            use_cache=True,
            max_new_tokens=128,
            eos_token_id=32001,
            pad_token_id=processor.tokenizer.pad_token_id
        )

    prompt_len = inputs["input_ids"].shape[1]
    decoded_text = processor.batch_decode(output[:, prompt_len:])[0]

    return decoded_text

# Define the Gradio interface
description = """Quick demonstration of the new Unum uForm-gen for image captioning. Upload an image to generate a detailed caption. Modify the Prompt to change the level of detail in the caption.

The model used in this app is available at [Hugging Face Model Hub](https://huggingface.co/unum-cloud/uform-gen) and the source code can be found on [GitHub](https://github.com/unum-cloud/uform)."""

iface = gr.Interface(
    fn=generate_caption,
    inputs=[gr.Image(type="pil", label="Upload Image"), gr.Textbox(label="Prompt", value="Describe the image in great detail")],
    outputs=gr.Textbox(label="Generated Caption"),
    description=description
)

# Launch the interface
iface.launch(share=True)

Writing /content/ugen-image-captioning-hf/app_cuda.py


In [ ]:
# cuda
%cd /content/ugen-image-captioning-hf
!python app.py